In [110]:
import pandas as pd
import numpy as np
from haversine import haversine
import networkx as nx
from tqdm import tqdm_notebook as tqdm
from lonelyboy.geospatial import preprocessing as gspp

df = pd.read_csv('4h_timeFrame_not_at_3_fing_am.csv')

/home/theo/anaconda3/envs/gsa/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [111]:
gspp.ts_from_str_datetime(df)

Droping 0 rows..


In [116]:
type(df.ts.iloc[1])

numpy.int64

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [117]:
df.head()

,id,mmsi,status,turn,speed,course,heading,lon,lat,ts,geom,velocity,datetime
0,1.884809e+07,227408710.0,0.0,0.0,0.000000,258.100000,511.000000,-4.490398,48.379186,1456817760,POINT (-4.4903984 48.3791865),0.013741,2016-03-01 07:36:00
1,1.758572e+07,228853000.0,7.0,-127.0,6.169685,200.775984,511.000000,-3.891684,47.732670,1456817760,POINT (-3.891684385826772 47.73266998031496),0.153449,2016-03-01 07:36:00
2,1.884808e+07,227686540.0,0.0,0.0,6.400000,199.300000,511.000000,-4.464395,48.327780,1456817760,POINT (-4.464395 48.32778),6.911790,2016-03-01 07:36:00
3,1.751793e+07,226216000.0,7.0,0.0,0.000000,196.639167,302.466667,-4.175265,47.835712,1456817760,POINT (-4.175265398333333 47.83571188333333),0.001396,2016-03-01 07:36:00
4,1.751790e+07,227270000.0,5.0,-127.0,0.000000,0.000000,511.000000,-4.478274,48.383153,1456817760,POINT (-4.478274466666667 48.38315333333333),0.003177,2016-03-01 07:36:00


In [4]:
def pairs_in_radius(df, diam=1000):
    res = []
    for ind_i, ind_j, val_i, val_j in nparray_combinations(df):
        dist = haversine(val_i, val_j)*1000
        if (dist<diam):
            res.append((ind_i,ind_j))   
    return res
            

In [5]:
def connected_edges(data):
    G = nx.Graph()
    G.add_edges_from(data)
    return [sorted(list(cluster)) for cluster in nx.find_cliques(G)]

In [6]:
def nparray_combinations(arr):
    for i in range(arr.shape[0]):
        for j in range(i+1, arr.shape[0]):
            yield i, j, arr[i,:], arr[j,:]

In [7]:
def eval_candidate(candidate, pairs):
    for pair in itertools.combinations(candidate,2):
        if pair not in pairs:
            return False
    return True

In [8]:
def translate(sets, sdf):
    return [sorted(tuple([sdf.iloc[point].mmsi for point in points])) for points in sets]

In [9]:
def circle_cluster(timeframe, min_cardinality = 2):
    pairs = pairs_in_radius(timeframe[['lon', 'lat']].values)
    full_sets = connected_edges(pairs)
    clusters = []
    for individual_sets in full_sets:
        # MAYBE FROM BIG TO SMALL INSTEAD OF SMALL TO BIG
        for j in range(min_cardinality,len(individual_sets)+1):
            for candidate in itertools.combinations(individual_sets,j):
                 if eval_candidate(candidate, pairs):
                        clusters.append(sorted(candidate))
    return clusters

In [10]:
def get_circular_clusters(timeframe):
    pairs = pairs_in_radius(timeframe[['lon', 'lat']].values)
    return connected_edges(pairs)
    

In [102]:
def find_existing_flocks(x, present, past, last_ts):
    # find the indices of past Dataframe where current cluster is subset of flock
    indcs = [set(x.clusters) < set(val) for val in past.loc[past.et==last_ts].clusters.values]
    # get the indices of the past dataframe where that occurs
    past.loc[(indcs)].index.tolist()

def replace_with_existing_flocks(x, present, to_keep, past):
    
    if to_keep.iloc[x.name]:
        if len(past.iloc[to_keep.iloc[x.name]])>1:
            raise Exception('len > 1, something is wrong')

        x.dur = past.iloc[to_keep.iloc[x.name]].dur.values[0] +1
        x.st = past.iloc[to_keep.iloc[x.name]].st.values[0]
    return x

def get_current_clusters(sdf, ts):
    present = pd.DataFrame([[tuple(val)] for (val) in translate(get_circular_clusters(sdf), sdf)], columns=['clusters'])
    present['st'] = present['et'] = ts
    present['dur'] = 1
    return present

def present_is_subset_of_past(present, past, last_ts):
    
    to_keep = present.apply(find_existing_flocks, args=(present,past,last_ts,) , axis=1)
#     print(to_keep)
    present = present.apply(replace_with_existing_flocks, args=(present,to_keep,past,), axis=1)
    
    new = present.merge(past,on='clusters', how='left',suffixes=['','tmp'], indicator=True)
    new = new[new['_merge']=='left_only'].drop(['_merge'],axis=1).dropna(axis=1)
    
    return new


def past_is_subset_or_set_of_present(present, past, ts, last_ts):
#     get if tuple of tmp1 is subset or equal of a row of tmp2
    ## Present > Past
    to_keep = past.apply(lambda x: (True in [set(x.clusters) <= set(val) for val in present.clusters.values]) and (x.et == last_ts), axis=1)

#     df_to_keep = past.loc[to_keep]

    past.loc[to_keep,'et'] = ts
    past.loc[to_keep,'dur']= past.loc[to_keep].dur.apply(lambda x : x+1)
    return past

def get_flocks(new_clusters, clusters_to_keep):
    return pd.concat([new_clusters,clusters_to_keep]).reset_index(drop=True)
    

In [118]:
log = []
keep = pd.DataFrame()

for ind, (ts, sdf) in tqdm(enumerate(df.groupby('ts')), total=df.datetime.nunique()):

    present = get_current_clusters(sdf, ts)
    if ind == 0:
        past = present
        last_ts = ts
        continue
        
    new_subsets = present_is_subset_of_past(present, past, last_ts)
    
    old_subsets_or_sets = past_is_subset_or_set_of_present(present, past, ts, last_ts)

    past = get_flocks(new_subsets, old_subsets_or_sets)

    past = past.loc[(past.et==ts) | (past.dur>10)] 
#     print(sum(new_subsets['clusters'].duplicated()),sum(old_subsets_or_sets['clusters'].duplicated()),sum(past['clusters'].duplicated()))
    last_ts = ts
    
# fn_keep = keep.drop_duplicates(['clusters'], keep='last')

In [119]:
past

,clusters,st,et,dur
0,"(227006750.0, 227008170.0, 227088590.0, 227298...",1456832160,1456832160,1
1,"(227222000.0, 227270000.0, 227303430.0, 227589...",1456832160,1456832160,1
2,"(227666970.0, 227690560.0, 412069000.0)",1456832160,1456832160,1
3,"(227006750.0, 227008170.0, 227088590.0, 227303...",1456832160,1456832160,1
5,"(227006750.0, 227222000.0, 227270000.0, 227303...",1456832100,1456832160,2
6,"(227006750.0, 227008170.0, 227270000.0, 227303...",1456832100,1456832160,2
7,"(227690560.0, 412069000.0)",1456832100,1456832160,2
8,"(227006750.0, 227008170.0, 227088590.0, 227303...",1456832040,1456832160,3
9,"(227636480.0, 636015106.0)",1456831980,1456832160,4
10,"(227113100.0, 227569380.0, 227650230.0)",1456831920,1456832160,5


In [101]:
new = present.merge(past,on='clusters', how='left',suffixes=['','tmp'], indicator=True)
new[new['_merge']=='left_only'].drop(['_merge'],axis=1).dropna(axis=1)

,clusters,st,et,dur
0,"(227002330.0, 227006750.0, 227008170.0, 227016...",2016-03-01 07:37:00,2016-03-01 07:37:00,2
41,1,1,1,1


In [76]:
present = present.append({'clusters':1, 'st':1,'et':1,'dur':1}, ignore_index=True)

In [70]:
present.dur = present.apply(lambda x: x.dur+1,axis=1)

In [95]:
present.at[0,'clusters'] = (227002330.0, 227006750.0, 227008170.0, 227016100.0, 227270000.0)

In [96]:
present

,clusters,st,et,dur
0,"(227002330.0, 227006750.0, 227008170.0, 227016...",2016-03-01 07:37:00,2016-03-01 07:37:00,2
1,"(227002330.0, 227006750.0, 227016100.0, 227222...",2016-03-01 07:37:00,2016-03-01 07:37:00,2
2,"(227322670.0, 227578460.0, 227611930.0, 227639...",2016-03-01 07:37:00,2016-03-01 07:37:00,2
3,"(227578460.0, 227611930.0, 227631450.0, 227639...",2016-03-01 07:37:00,2016-03-01 07:37:00,2
4,"(227322670.0, 227639660.0, 227686540.0)",2016-03-01 07:37:00,2016-03-01 07:37:00,2
5,"(227142200.0, 227162950.0, 227941000.0)",2016-03-01 07:37:00,2016-03-01 07:37:00,2
6,"(227640710.0, 227654220.0)",2016-03-01 07:37:00,2016-03-01 07:37:00,2
7,"(227315110.0, 227666970.0)",2016-03-01 07:37:00,2016-03-01 07:37:00,2
8,"(227641920.0, 227686540.0)",2016-03-01 07:37:00,2016-03-01 07:37:00,2
9,"(227588970.0, 227590030.0)",2016-03-01 07:37:00,2016-03-01 07:37:00,2


In [213]:
present

,clusters,st,et,dur
0,"(226263000.0, 227002330.0, 227006750.0, 227008...",2016-03-01 07:37:00,2016-03-01 07:37:00,1
1,"(227002330.0, 227006750.0, 227016100.0, 227222...",2016-03-01 07:37:00,2016-03-01 07:37:00,1
2,"(227322670.0, 227578460.0, 227611930.0, 227639...",2016-03-01 07:37:00,2016-03-01 07:37:00,1
3,"(227578460.0, 227611930.0, 227631450.0, 227639...",2016-03-01 07:37:00,2016-03-01 07:37:00,1
4,"(227322670.0, 227639660.0, 227686540.0)",2016-03-01 07:37:00,2016-03-01 07:37:00,1
5,"(227142200.0, 227162950.0, 227941000.0)",2016-03-01 07:37:00,2016-03-01 07:37:00,1
6,"(227640710.0, 227654220.0)",2016-03-01 07:37:00,2016-03-01 07:37:00,1
7,"(227315110.0, 227666970.0)",2016-03-01 07:37:00,2016-03-01 07:37:00,1
8,"(227641920.0, 227686540.0)",2016-03-01 07:37:00,2016-03-01 07:37:00,1
9,"(227588970.0, 227590030.0)",2016-03-01 07:37:00,2016-03-01 07:37:00,1


In [227]:
present = present.apply(replace_with_existing_flocks, args=(new_clusters,past,),axis=1)

clusters    (227142200.0, 227162950.0, 227941000.0)
st                              2016-03-01 07:38:00
et                              2016-03-01 07:38:00
dur                                               1
Name: 0, dtype: object
clusters    (227142200.0, 227162950.0, 227941000.0)
st                              2016-03-01 07:38:00
et                              2016-03-01 07:38:00
dur                                               1
Name: 0, dtype: object
clusters    (227303430.0, 227632830.0)
st                 2016-03-01 07:38:00
et                 2016-03-01 07:38:00
dur                                  1
Name: 1, dtype: object
clusters    (227003050.0, 228064900.0)
st                 2016-03-01 07:38:00
et                 2016-03-01 07:38:00
dur                                  1
Name: 2, dtype: object
clusters    (227640710.0, 227654220.0)
st                 2016-03-01 07:38:00
et                 2016-03-01 07:38:00
dur                                  1
Name: 3, dtype: object
c

In [228]:
present

,clusters,st,et,dur
0,"(227142200.0, 227162950.0, 227941000.0)",2016-03-01 07:38:00,2016-03-01 07:38:00,1
1,"(227303430.0, 227632830.0)",2016-03-01 07:38:00,2016-03-01 07:38:00,1
2,"(227003050.0, 228064900.0)",2016-03-01 07:38:00,2016-03-01 07:38:00,1
3,"(227640710.0, 227654220.0)",2016-03-01 07:38:00,2016-03-01 07:38:00,1
4,"(227322670.0, 227639660.0, 227650230.0, 227686...",2016-03-01 07:38:00,2016-03-01 07:38:00,1
5,"(227322670.0, 227578460.0, 227611930.0, 227631...",2016-03-01 07:38:00,2016-03-01 07:38:00,1
6,"(249297000.0, 477115900.0)",2016-03-01 07:38:00,2016-03-01 07:38:00,1
7,"(227322670.0, 227589520.0)",2016-03-01 07:38:00,2016-03-01 07:38:00,1
8,"(228064900.0, 228186700.0)",2016-03-01 07:38:00,2016-03-01 07:38:00,1
9,"(227315110.0, 227666970.0)",2016-03-01 07:38:00,2016-03-01 07:38:00,1
